In [ ]:
# import necessary packages
import numpy as np
import pandas as pd

# path to data we just downloaded
# if you are interested in how modelpoints were created,
# check out the `generating_modelpoints.ipynb` file in the repo
DATA_XLS = "./dstoolkit/model_point.xlsx"

# ingesting our data into a dataframe (df)

### Creating & editting spreadsheets from CLI with `openpyxl` & 'xlwings`

In [6]:
%pip install openpyxl appscript psutil xlwings

Note: you may need to restart the kernel to use updated packages.


In [16]:
import xlwings as xw
import pandas as pd

# with xw one can open new workbook in Interactve mode
book = xw.Book()

# `xwings` Interactive Mode

In [21]:
# to list the sheets in the workbook in xlwings
book.sheets

Sheets([<Sheet [Book8]Sheet1>])

In [25]:
# We wish to then change the values of cells in our workbook sheet
print(book.sheets[0]["A1"].value)

None


In [34]:
# Assigning values to a cell
# Notice the mismatched dimensionality
# Impact is similar to pasting table into a cell in Excel
book.sheets[0]["A1"].value = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})

We obsere, not only does our `pd.DataFrame` expands beyond `"A1"`,`"A1"` is not filled with the `pd.DataFrame` 

However if we assign a  single value to `"A1"` the value of `"A1"` itself is set. Seems to be related to the dimensionality mismatch. Is almost like pasting a table to a cell in Excel.

In [30]:
print(book.sheets[0]["A1"].value)

None
